In [1]:
import numpy as np
import os

np.random.seed(42)

DATA_PATH = 'dataset'

In [2]:
from scipy.io import arff

def load_pulsar_csv(path = DATA_PATH):
    csv_path = os.path.join(path, 'HTRU_2.csv')
    return np.loadtxt(csv_path, delimiter=',', dtype=np.float32)

def load_pulsar_arff(path = DATA_PATH):
    arff_path = os.path.join(path, 'HTRU_2.arff')
    return arff.loadarff(arff_path)

In [3]:
pulsars = load_pulsar_csv()

In [4]:
import numpy as np

def split_train_dataset(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(test_ratio * len(data))
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data[train_indices,:], data[test_indices,:]

# Use hash of identifier to decide if instance goes into train or test set

In [5]:
# Save Model
import os

MODELS_DIR = "models"

def save_model(model, name, acc=None):
    name += str(model.input.shape[1])
    for layer in model.layers:
        name += "-" + str(layer.output.shape[1:])
    
    name += "_" + (("%.2f" % acc) if acc is not None else "")
    path = os.path.join(MODELS_DIR, name + ".h5")
    model.save(path)

In [6]:
train_set, test_set = split_train_dataset(pulsars, 0.2)

X_train, Y_train = train_set[:, :-1], train_set[:, -1]
X_test, Y_test = test_set[:, :-1], test_set[:, -1]

In [7]:
from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, Reshape

# Create Model
input_dimension = np.size(X_train, axis=1)

def create_model_cnn():
    inputs = Input(shape=(input_dimension,), dtype='float32')
    reshape = Reshape((input_dimension,1))(inputs) # reshape input (?,8) to conv input (?,8,1)
    conv_0 = Conv1D(64, kernel_size=6, activation='relu')(reshape)
    pool = MaxPooling1D()(conv_0)
    flatten = Flatten()(pool)
    dropout = Dropout(0.2)(flatten)
    output = Dense(1, activation='sigmoid')(dropout)
    # Compile model
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [8]:
model = create_model_cnn()

In [9]:
# Fit the Model
model.fit(X_train, Y_train, epochs=150, batch_size=15, validation_data=[X_test, Y_test])

Train on 14319 samples, validate on 3579 samples
Epoch 1/150
14319/14319 [==============================] - 3s 242us/step - loss: 0.1570 - acc: 0.9610 - val_loss: 0.0888 - val_acc: 0.9737
Epoch 2/150
14319/14319 [==============================] - 3s 208us/step - loss: 0.0963 - acc: 0.9736 - val_loss: 0.0882 - val_acc: 0.9743
Epoch 3/150
14319/14319 [==============================] - 3s 216us/step - loss: 0.0952 - acc: 0.9726 - val_loss: 0.0900 - val_acc: 0.9740
Epoch 4/150
14319/14319 [==============================] - 3s 216us/step - loss: 0.0896 - acc: 0.9749 - val_loss: 0.0868 - val_acc: 0.9751
Epoch 5/150
14319/14319 [==============================] - 3s 219us/step - loss: 0.0915 - acc: 0.9744 - val_loss: 0.0944 - val_acc: 0.9743
Epoch 6/150
14319/14319 [==============================] - 3s 216us/step - loss: 0.0879 - acc: 0.9758 - val_loss: 0.0851 - val_acc: 0.9763
Epoch 7/150
14319/14319 [==============================] - 3s 235us/step - loss: 0.0896 - acc: 0.9743 - val_loss: 0.0

14319/14319 [==============================] - 3s 239us/step - loss: 0.0808 - acc: 0.9764 - val_loss: 0.0745 - val_acc: 0.9768
Epoch 60/150
14319/14319 [==============================] - 3s 218us/step - loss: 0.0822 - acc: 0.9760 - val_loss: 0.0789 - val_acc: 0.9779
Epoch 61/150
14319/14319 [==============================] - 3s 228us/step - loss: 0.0827 - acc: 0.9765 - val_loss: 0.0751 - val_acc: 0.9790
Epoch 62/150
14319/14319 [==============================] - 3s 221us/step - loss: 0.0811 - acc: 0.9768 - val_loss: 0.0774 - val_acc: 0.9765
Epoch 63/150
14319/14319 [==============================] - 3s 236us/step - loss: 0.0804 - acc: 0.9766 - val_loss: 0.0748 - val_acc: 0.9796
Epoch 64/150
14319/14319 [==============================] - 3s 232us/step - loss: 0.0799 - acc: 0.9765 - val_loss: 0.0737 - val_acc: 0.9779
Epoch 65/150
14319/14319 [==============================] - 3s 219us/step - loss: 0.0811 - acc: 0.9757 - val_loss: 0.0733 - val_acc: 0.9779
Epoch 66/150
14319/14319 [=======

14319/14319 [==============================] - 3s 216us/step - loss: 0.0787 - acc: 0.9765 - val_loss: 0.0730 - val_acc: 0.9790
Epoch 118/150
14319/14319 [==============================] - 3s 218us/step - loss: 0.0806 - acc: 0.9772 - val_loss: 0.0735 - val_acc: 0.9788
Epoch 119/150
14319/14319 [==============================] - 3s 212us/step - loss: 0.0777 - acc: 0.9770 - val_loss: 0.0729 - val_acc: 0.9788
Epoch 120/150
14319/14319 [==============================] - 3s 218us/step - loss: 0.0799 - acc: 0.9759 - val_loss: 0.0735 - val_acc: 0.9785
Epoch 121/150
14319/14319 [==============================] - 3s 213us/step - loss: 0.0796 - acc: 0.9767 - val_loss: 0.0832 - val_acc: 0.9774
Epoch 122/150
14319/14319 [==============================] - 3s 215us/step - loss: 0.0801 - acc: 0.9756 - val_loss: 0.0722 - val_acc: 0.9796
Epoch 123/150
14319/14319 [==============================] - 3s 244us/step - loss: 0.0783 - acc: 0.9767 - val_loss: 0.0706 - val_acc: 0.9790
Epoch 124/150
14319/14319 [

In [10]:
from sklearn.metrics import classification_report

# Classification metrics on test data
predictions = model.predict(X_test)
Y_test_predictions = [int(y + 0.5) for y in predictions]
print(classification_report(Y_test, Y_test_predictions))

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99      3259
        1.0       0.94      0.81      0.87       320

avg / total       0.98      0.98      0.98      3579



In [11]:
# Evaluate the Model
scores = model.evaluate(X_test, Y_test)
print("Accuracy:", scores[1] * 100)

save_model(model, 'pulsar_CNN_', scores[1] * 100)

3579/3579 [==============================] - 0s 30us/step
Accuracy: 97.8485610389
